<a href="https://colab.research.google.com/github/MinsooKwak/Study/blob/main/DA/LTV_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LTV

- LTV (Lifetime Value, CLV)
  - 고객 생애 가치
  - 유저가 서비스를 사용하는 기간 동안 창출하는 매출

- 실무 사례
  1. 신규 고객 획득 비용 (CAC) 기준 설정
  2. 효율적 마케팅 채널 파악해 예산 분배

- LTV 계산
  - 계산 방법 다양하고 까다로움. 변동성 큼
  - 전통적
    - LTV = ARPU * 리텐션
      - ARPU (유저당 주문고객) = 매출 / 유저 수
    - 그룹을 코호트로 쪼개 LTV 분석

## LTV 분석

1. 분석 주제 : 새벽 배송 플랫폼 핵심 고객 연령층
2. 측정 인원 : 23년 1월 첫 주문 고객 중 10~50대까지 연령별 100명 (총 500)
3. 분석 방법 : 23년 1월 첫 주문 고객의 6개월 간 연령대별 인당 누적 LTV 계산

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
first_order_table = pd.read_csv('/content/drive/MyDrive/Study/DA/Data/LTV/first_ord_table_ltv.csv')
order_master = pd.read_csv('/content/drive/MyDrive/Study/DA/Data/LTV/order_master_ltv.csv')

In [4]:
print(f'총 {first_order_table.shape[0]}개 데이터셋')
first_order_table.head(3)

총 500개 데이터셋


,mem_no,first_ord_dt,age_range
0,1,2023-01-13,20대
1,2,2023-01-10,40대
2,3,2023-01-16,40대


In [5]:
print(f'총 {order_master.shape[0]}개 데이터셋')
order_master.head(3)

총 3268개 데이터셋


,mem_no,ord_dt,order_amount
0,1,2023-01-13,30580
1,2,2023-01-10,31294
2,3,2023-01-16,28260


In [6]:
df = pd.merge(first_order_table, order_master, on='mem_no')
df.head(3)

,mem_no,first_ord_dt,age_range,ord_dt,order_amount
0,1,2023-01-13,20대,2023-01-13,30580
1,2,2023-01-10,40대,2023-01-10,31294
2,3,2023-01-16,40대,2023-01-16,28260


In [7]:
df.shape[0]

3268

- ARPU, Retention 구해야함
  - Retention 구하기 위해 6개월 구간 나눠야 함 (코호트)
    - 첫 주문 월(M0) : 첫 주문 일 = 주문 일
    - 둘째 주문 월(M1) : (첫 주문 월 < ord_dt) & (order_dt <= first_order_table + 1달)
    - 셋째 주문 월(M2) : (둘째 주문 월 < ord_dt) & (order_dt <= first_order_table + 2달)
    - 넷째 주문 월(M3) : (셋째 주문 월 < ord_dt) & (ord_df <= first_order_table +3달)
    - 다섯째 주문 월(M4) : (넷째 주문 월 < ord_dt) & (ord_df <= first_order_table + 4달)
    - 나머지 over_5m
  - 연령대별, 구간 별 sample 수 count

In [8]:
from datetime import datetime, timedelta

In [9]:
# datetime으로 변경
df.first_ord_dt = pd.to_datetime(df.first_ord_dt)
df.ord_dt = pd.to_datetime(df.ord_dt)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3268 entries, 0 to 3267
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   mem_no        3268 non-null   int64         
 1   first_ord_dt  3268 non-null   datetime64[ns]
 2   age_range     3268 non-null   object        
 3   ord_dt        3268 non-null   datetime64[ns]
 4   order_amount  3268 non-null   int64         
dtypes: datetime64[ns](2), int64(2), object(1)
memory usage: 282.2+ KB


In [10]:
def time_day(x):
  x = str(x)
  x =x.split(' ')[0]
  return x

In [11]:
df['order_len'] = (df['ord_dt'] - df['first_ord_dt']).apply(time_day)
df['order_len'] = df['order_len'].astype(int)
df.head(3)

,mem_no,first_ord_dt,age_range,ord_dt,order_amount,order_len
0,1,2023-01-13,20대,2023-01-13,30580,0
1,2,2023-01-10,40대,2023-01-10,31294,0
2,3,2023-01-16,40대,2023-01-16,28260,0


In [12]:
def purchase_len(x):
  if x == 0:
    return 'M0'
  elif 0 < x <= 30:
    return 'M1'
  elif 30 < x <= 60:
    return 'M2'
  elif 60 < x <= 90:
    return 'M3'
  elif 90 < x <= 120:
    return 'M4'
  elif 120 < x <= 150:
    return 'M5'
  else:
    return 'over_M5'

In [13]:
df['group'] = df['order_len'].apply(purchase_len)
df.head(3)

,mem_no,first_ord_dt,age_range,ord_dt,order_amount,order_len,group
0,1,2023-01-13,20대,2023-01-13,30580,0,M0
1,2,2023-01-10,40대,2023-01-10,31294,0,M0
2,3,2023-01-16,40대,2023-01-16,28260,0,M0


In [14]:
df.group.value_counts(dropna=False)

M1         751
over_M5    713
M0         527
M2         388
M4         300
M3         295
M5         294
Name: group, dtype: int64

- 연령대별로 첫 주문로부터 기간을 나눠놨음
  - first order table의 order_amount를 merge 했음
- 연령대별, 기간 별 sample 수 count 해야 함

In [15]:
df_rearrange = pd.DataFrame(df.groupby(['mem_no','first_ord_dt','age_range'])['mem_no'].count())
df_rearrange.columns = ['amount']
df_rearrange.reset_index(inplace=True)
df_rearrange

,mem_no,first_ord_dt,age_range,amount
0,1,2023-01-13,20대,1
1,2,2023-01-10,40대,1
2,3,2023-01-16,40대,1
3,4,2023-01-01,50대,6
4,5,2023-01-27,30대,2
...,...,...,...,...
495,496,2023-01-21,30대,2
496,497,2023-01-21,20대,6
497,498,2023-01-02,10대,1
498,499,2023-01-09,10대,1


In [29]:
# ARPU = 매출 / 인원 수 (average도 동일)
df2 = pd.DataFrame(df.groupby(['age_range','group']).agg({'order_amount':'mean',
                                                          'mem_no':'count'}))
df2.columns = ['ARPU','sample_cnt']
df2.reset_index(inplace=True)
df2.ARPU = round(df2.ARPU,0).astype(int)
df2 = df2[['age_range','group','sample_cnt','ARPU']]
df2.head(10)

,age_range,group,sample_cnt,ARPU
0,10대,M0,104,28373
1,10대,M1,79,21995
2,10대,M2,33,28184
3,10대,M3,36,40543
4,10대,M4,27,26371
5,10대,M5,32,31252
6,10대,over_M5,67,32329
7,20대,M0,105,29643
8,20대,M1,139,25752
9,20대,M2,77,52719


- 리텐션 구하기 위해 한 번 더 집계 진행
  - sample_cnt(sample size)를 최초 샘플 사이즈로 나누는 것

In [31]:
max_sample = pd.DataFrame(df2.groupby(['age_range'])['sample_cnt'].max())
max_sample.columns = ['max_count']
max_sample.reset_index(inplace=True)
max_sample

,age_range,max_count
0,10대,104
1,20대,139
2,30대,235
3,40대,200
4,50대,174


In [38]:
df3 = pd.merge(df2, max_sample)
df3.head(10)

,age_range,group,sample_cnt,ARPU,max_count
0,10대,M0,104,28373,104
1,10대,M1,79,21995,104
2,10대,M2,33,28184,104
3,10대,M3,36,40543,104
4,10대,M4,27,26371,104
5,10대,M5,32,31252,104
6,10대,over_M5,67,32329,104
7,20대,M0,105,29643,139
8,20대,M1,139,25752,139
9,20대,M2,77,52719,139


In [ ]:
# sample_cnt / 연령별 max(sample_cnt)

In [41]:
df3['retention'] = df3['sample_cnt'] / df3['max_count']
df3['retention'] = round(df3.retention, 2)
df3.head()

,age_range,group,sample_cnt,ARPU,max_count,retention
0,10대,M0,104,28373,104,1.00
1,10대,M1,79,21995,104,0.76
2,10대,M2,33,28184,104,0.32
3,10대,M3,36,40543,104,0.35
4,10대,M4,27,26371,104,0.26


In [43]:
df3['LTV'] = df3['ARPU'] * df3['retention']
df3.head(10)

,age_range,group,sample_cnt,ARPU,max_count,retention,LTV
0,10대,M0,104,28373,104,1.00,28373.00
1,10대,M1,79,21995,104,0.76,16716.20
2,10대,M2,33,28184,104,0.32,9018.88
3,10대,M3,36,40543,104,0.35,14190.05
4,10대,M4,27,26371,104,0.26,6856.46
5,10대,M5,32,31252,104,0.31,9688.12
6,10대,over_M5,67,32329,104,0.64,20690.56
7,20대,M0,105,29643,139,0.76,22528.68
8,20대,M1,139,25752,139,1.00,25752.00
9,20대,M2,77,52719,139,0.55,28995.45


In [48]:
# 10대 LTV
LTV_10 = df3[(df3.age_range=='10대') & (df3.group != 'over_M5')]
print(f'23년도 1월에 첫 주문한 10대가 인당 6개월간 {int(LTV_10.LTV.sum())}원 정도 사용한다')
LTV_10

23년도 1월에 첫 주문한 10대가 인당 6개월간 84842원 정도 사용한다


,age_range,group,sample_cnt,ARPU,max_count,retention,LTV
0,10대,M0,104,28373,104,1.00,28373.00
1,10대,M1,79,21995,104,0.76,16716.20
2,10대,M2,33,28184,104,0.32,9018.88
3,10대,M3,36,40543,104,0.35,14190.05
4,10대,M4,27,26371,104,0.26,6856.46
5,10대,M5,32,31252,104,0.31,9688.12


In [49]:
# 20대 LTV
LTV_20 = df3[(df3.age_range=='20대') & (df3.group != 'over_M5')]
print(f'23년도 1월에 첫 주문한 20대가 인당 6개월간 {int(LTV_20.LTV.sum())}원 정도 사용한다')
LTV_20

23년도 1월에 첫 주문한 20대가 인당 6개월간 114740원 정도 사용한다


,age_range,group,sample_cnt,ARPU,max_count,retention,LTV
7,20대,M0,105,29643,139,0.76,22528.68
8,20대,M1,139,25752,139,1.00,25752.00
9,20대,M2,77,52719,139,0.55,28995.45
10,20대,M3,52,28343,139,0.37,10486.91
11,20대,M4,45,36823,139,0.32,11783.36
12,20대,M5,56,37986,139,0.40,15194.40


In [50]:
# 30대 LTV
LTV_30 = df3[(df3.age_range=='30대') & (df3.group != 'over_M5')]
print(f'23년도 1월에 첫 주문한 30대가 인당 6개월간 {int(LTV_30.LTV.sum())}원 정도 사용한다')
LTV_30

23년도 1월에 첫 주문한 30대가 인당 6개월간 109764원 정도 사용한다


,age_range,group,sample_cnt,ARPU,max_count,retention,LTV
14,30대,M0,106,35163,235,0.45,15823.35
15,30대,M1,235,43696,235,1.00,43696.00
16,30대,M2,74,41913,235,0.31,12993.03
17,30대,M3,72,43715,235,0.31,13551.65
18,30대,M4,59,40954,235,0.25,10238.50
19,30대,M5,71,44874,235,0.30,13462.20


In [51]:
# 40대 LTV
LTV_40 = df3[(df3.age_range=='40대') & (df3.group != 'over_M5')]
print(f'23년도 1월에 첫 주문한 40대가 인당 6개월간 {int(LTV_40.LTV.sum())}원 정도 사용한다')
LTV_40

23년도 1월에 첫 주문한 40대가 인당 6개월간 101309원 정도 사용한다


,age_range,group,sample_cnt,ARPU,max_count,retention,LTV
21,40대,M0,109,31157,200,0.55,17136.35
22,40대,M1,136,34586,200,0.68,23518.48
23,40대,M2,98,38323,200,0.49,18778.27
24,40대,M3,65,38428,200,0.32,12296.96
25,40대,M4,76,38513,200,0.38,14634.94
26,40대,M5,71,41513,200,0.36,14944.68


In [52]:
# 50대 LTV
LTV_50 = df3[(df3.age_range=='50대') & (df3.group != 'over_M5')]
print(f'23년도 1월에 첫 주문한 50대가 인당 6개월간 {int(LTV_50.LTV.sum())}원 정도 사용한다')
LTV_50

23년도 1월에 첫 주문한 50대가 인당 6개월간 133374원 정도 사용한다


,age_range,group,sample_cnt,ARPU,max_count,retention,LTV
28,50대,M0,103,29242,174,0.59,17252.78
29,50대,M1,162,34697,174,0.93,32268.21
30,50대,M2,106,41364,174,0.61,25232.04
31,50대,M3,70,45461,174,0.40,18184.40
32,50대,M4,93,44951,174,0.53,23824.03
33,50대,M5,64,44900,174,0.37,16613.00


In [59]:
age = ['10대','20대','30대','40대','50대']
LTV_f = [int(LTV_10.LTV.sum()), int(LTV_20.LTV.sum()), int(LTV_30.LTV.sum()), int(LTV_40.LTV.sum()), int(LTV_50.LTV.sum())]

df_f = pd.DataFrame(data=[LTV_f],
                    columns=age)

df_f

,10대,20대,30대,40대,50대
0,84842,114740,109764,101309,133374


- 23년 1월 가입대 중 첫 주문 이후 5개월까지 50대가 가장 높은 LTV
- 2번째로 20대
- 3번째로 30대
- 4번째로 40대
- 마지막 10대 순